In [2]:
from api.db import stat_db, crawl_db, CrawlCollections, StatCollections
import spacy
from spacy import displacy
from spacy.matcher import Matcher
nlp = spacy.load('en_core_web_sm')
matcher = Matcher(nlp.vocab)

In [ ]:
doc = nlp(u'Apple is looking at buying U.K. startup for $1 billion')
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
          token.shape_, token.is_alpha, token.is_stop)

In [ ]:

doc = nlp(u'Autonomous cars shift insurance liability toward manufacturers')
for chunk in doc.noun_chunks:
    print(chunk.root.dep_)


In [ ]:

matcher = Matcher(nlp.vocab)
# add match ID "HelloWorld" with no callback and one pattern
pattern = [{'LOWER': 'hello'}, {'IS_PUNCT': True}, {'LOWER': 'world'}]
matcher.add('HelloWorld', None, pattern)

doc = nlp(u'Hello, world! Hello world!')
matches = matcher(doc)
print(matches)


In [4]:

# Get the ID of the 'EVENT' entity type. This is required to set an entity.
EVENT = nlp.vocab.strings['EVENT']


def add_event_ent(matcher, doc, i, matches):
    # Get the current match and create tuple of entity label, start and end.
    # Append entity to the doc's entity. (Don't overwrite doc.ents!)
    match_id, start, end = matches[i]
    doc.ents += ((EVENT, start, end),)


matcher.add('GoogleIO', add_event_ent,
            [{'ORTH': 'Google'}, {'UPPER': 'I'}, {'ORTH': '/'}, {'UPPER': 'O'}],
            [{'ORTH': 'Google'}, {'UPPER': 'I'}, {'ORTH': '/'}, {'UPPER': 'O'}, {'IS_DIGIT': True}])


In [11]:
matched_sents = [] # collect data of matched sentences to be visualized


def collect_sents(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    span = doc[start : end] # matched span
    sent = span.sent # sentence containing matched span
    # append mock entity for match in displaCy style to matched_sents
    # get the match span by ofsetting the start and end of the span with the
    # start and end of the sentence in the doc
    match_ents = [{'start': span.start_char - sent.start_char,
                   'end': span.end_char - sent.start_char,
                   'label': 'MATCH'}]
    matched_sents.append({'text': sent.text, 'ents': match_ents })
    

pattern = [{'LOWER': 'wallet'}, {'POS': 'ADV', 'OP': '*'},
           {'POS': 'ADJ', 'OP': '*'}]
matcher.add('FacebookIs', collect_sents, pattern) # add pattern

cursor = crawl_db[CrawlCollections.BTT_COMMENT].find({'data.link_id': '1883902'}).limit(100)
for item in cursor:
    text = item['data']['content'] 
    matches = matcher(nlp(text)) # match on your text
    
print(matched_sents)

In [3]:
text = '1.We upload clip introduce from Co-founder and in another day we have Live! on facebook fanpage please see and subscribe.https://www.youtube.com/watch?v=DxQ8ODgdPLgIf you want to Receive Free EST Coin Please Join My Bounty Program1.Translate Whitepaper : 1000 ESTHow to join bounty1.Send This data to realestatecoins@gmail.com       1.1 Your Name - Surname       1.2 Your County       1.3 Your Ether wallet Address       1.4 Your jobs do you want.       1.5 Link profile bitcointalk2.If your Request is  You will see Your name in Excel file and See Token EST that will you receivehttps://docs.google.com/spreadsheets/d/1LMJcTG-P8YoHKixpwd7CNfNQ_f102x5of_X47iRNNRE/edit?usp=sharingBitcointalk BOUNTY PROGRAM https://bitcointalk.org/index.php?topic=2315043.new#newIf you want to Receive Free EST Coin Please Join My Bounty Program1.Create and Manage Campaign BOUNTY Project at bitcointalk.org : 5000 EST - 10000 ESTHow to join bounty1.Send This data to realestatecoins@gmail.com       1.1 Your Name - Surname       1.2 Your County       1.3 Your Ether wallet Address       1.4 Your jobs do you want.       1.5 Link profile bitcointalk2.If your Request is  You will see Your name in Excel file and See Token EST that will you receivehttps://docs.google.com/spreadsheets/d/1LMJcTG-P8YoHKixpwd7CNfNQ_f102x5of_X47iRNNRE/edit?usp=sharingCoinhills add estatecoion.fund to list >>> https://www.coinhills.com/ico/view/estatecoin/Today we have a plan to announce BOUNTY Program 3 about Signature bitcointalk.org that every member can join and We have discuss about how to make more value to EST coin we have a plan to make a mini real estate project in pattaya that make Profit more 3% per month (I my Idea is about 5-10% per month or more) and On pre-ico that investor invest we maybe invite to stay in my estate to see real estate too. Please follow us.If you want to Receive Free EST Coin Please Join My Bounty ProgramJR.Member  1 post x 0.1 EST / postMEMBER 1 post x 0.15 EST / postFull member 1 post x 0.20 EST / postSR Member 1 post x 0.25 EST / postHERO / LEGENDARY 1 post x 0.30 EST / postHow to join bounty1.Send This data to realestatecoins@gmail.com       1.1 Your username on bitcointalk       1.2 Your Country       1.3 Your Ether wallet Address       1.4 Your jobs do you want. (Signature Profile)       1.5 Link profile bitcointalk2.If your Request is  You will see Your name in Excel file and See Token EST that will you receivehttps://docs.google.com/spreadsheets/d/1LMJcTG-P8YoHKixpwd7CNfNQ_f102x5of_X47iRNNRE/edit?usp=sharing3.Rule You need to put the signature. Throughout the pre-ico and ico openings, we will randomly check. If we find fraud, we will not send you EST and if the project is still open ico you are promoted. We will give you a return.'
doc = nlp(text)
for entity in doc.ents:
    print(entity.text, entity.label_)